In [15]:
import os
import posixpath
import xarray as xr
import rioxarray
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import pandas as pd
import json

# parallel computing
from dask.distributed import Client

# personnal packages
import optim.space_rejection as sr
import geometry.internal_orientation as gio
import geometry.external_orientation as geo

In [16]:
images_root = 'D:/OneDrive/Documents/Cours/4A/SFE/data/KH-5_ARGON_images'
products = []

for x in os.listdir(images_root):
    if os.path.isdir(posixpath.join(images_root, x)):
        products.append(x)
        
products

['DS09034A007MC018',
 'DS09034A007MC019',
 'DS09034A008MC019',
 'DS09034A008MC020',
 'DS09034A008MC021',
 'DS09034A008MC022',
 'DS09058A024MC012',
 'DS09058A024MC013']

In [17]:
all_params_path = posixpath.join(images_root, "images_params.json")
if os.path.exists(posixpath.join(images_root, "images_params.json")) == False:
    all_params = {
        "description": "Internal and external orientation parameters for KH-5 ARGON images"
    }
    with open(all_params_path, "w") as f:
        json.dump(all_params, f, indent=4)

In [18]:
rasters = [
    'DS09058A024MC012',
    'DS09058A024MC013'
]

all_GCPs = []

for i_raster, raster in enumerate(rasters):
    # raster_path = posixpath.join(images_root, raster, raster + '_a.tif')
    GCPs_path = posixpath.join(images_root, raster, 'GCP.points')
    GCPs_i = pd.read_csv(GCPs_path, encoding="windows-1252", skiprows=1)
    GCPs_i.columns = ["lon", "lat", "x_img", "y_img", "enable", "dX", "dY", "residual"]
    GCPs_i.drop(columns=["enable", "dX", "dY", "residual"], inplace=True)
    GCPs_i.loc[:, "y_img"] = - GCPs_i.loc[:, "y_img"]
    GCPs_i.loc[:, "image"] = raster
    all_GCPs.append(GCPs_i)
    
GCPs = pd.concat(all_GCPs, ignore_index=True)
print(len(GCPs), "GCPs found for", len(rasters), "rasters")
GCPs.head()

212 GCPs found for 2 rasters


,lon,lat,x_img,y_img,image
0,15.669701,78.296340,10090.231823,6378.932688,DS09058A024MC012
1,15.822687,78.257831,10106.594368,6190.345399,DS09058A024MC012
2,15.951993,78.240516,10083.782353,6076.763062,DS09058A024MC012
3,16.027331,78.219240,10101.936417,5980.737615,DS09058A024MC012
4,16.241150,78.218166,9995.579875,5857.839376,DS09058A024MC012


In [19]:
DEM_root = 'D:/OneDrive/Documents/Cours/4A/SFE/data/NPI_DEMs/NP_S0_DTM20'
file = posixpath.join(DEM_root, "S0_DTM20_EPSG4326.tif")

DEM = rioxarray.open_rasterio(file, chunks=True)
DEM = DEM.rename({"x": "lon", "y": "lat"})
DEM.rio.set_nodata(0., inplace=True)
DEM = DEM.where(DEM < 3e38, 0.)
DEM

<xarray.DataArray (band: 1, lat: 15890, lon: 56948)> Size: 4GB
dask.array<where, shape=(1, 15890, 56948), dtype=float32, chunksize=(1, 5760, 5760), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int32 4B 1
  * lon          (lon) float64 456kB 9.498 9.498 9.499 ... 33.84 33.84 33.84
  * lat          (lat) float64 127kB 80.87 80.87 80.87 ... 74.08 74.08 74.07
    spatial_ref  int32 4B 0
Attributes: (12/15)
    RepresentationType:        ATHEMATIC
    STATISTICS_COVARIANCES:    34445.33087394282
    STATISTICS_MAXIMUM:        1629.8674316406
    STATISTICS_MEAN:           75.107037767815
    STATISTICS_MEDIAN:         -0,920757
    STATISTICS_MINIMUM:        -0.79490000009537
    ...                        ...
    AREA_OR_POINT:             Area
    STATISTICS_APPROXIMATE:    YES
    STATISTICS_VALID_PERCENT:  72.84
    _FillValue:                3.4e+38
    scale_factor:              1.0
    add_offset:                0.0

In [20]:
h = DEM.interp(lat=("z", GCPs.lat.values), lon=("z", GCPs.lon.values), method="nearest").compute()
GCPs.loc[:, "h"] = h.values.reshape(-1)
GCPs = GCPs.loc[:, ["image", "lon", "lat", "h", "x_img", "y_img"]]
GCPs.head()

,image,lon,lat,h,x_img,y_img
0,DS09058A024MC012,15.669701,78.296340,475.462158,10090.231823,6378.932688
1,DS09058A024MC012,15.822687,78.257831,902.182007,10106.594368,6190.345399
2,DS09058A024MC012,15.951993,78.240516,857.888794,10083.782353,6076.763062
3,DS09058A024MC012,16.027331,78.219240,833.358032,10101.936417,5980.737615
4,DS09058A024MC012,16.241150,78.218166,1070.018433,9995.579875,5857.839376


In [21]:
# geocentric cartesian coordinates
x_geo, y_geo, z_geo = geo.geodetic_to_geocentric_cartesian_coordinates(GCPs.lat.values *np.pi/180, GCPs.lon.values *np.pi/180, GCPs.h.values)
GCPs.loc[:, ["x_geo", "y_geo", "z_geo"]] = np.array([x_geo, y_geo, z_geo]).T

for i_raster, raster in enumerate(rasters):
    with open(all_params_path, "r") as f:
        all_params = json.load(f)
        
    # local cartesian coordinates
    lat_c, lon_c = all_params[raster]["external_orientation"]["lat_c"], all_params[raster]["external_orientation"]["lon_c"]
    x_geo, y_geo, z_geo = GCPs.loc[GCPs.image == raster, "x_geo"], GCPs.loc[GCPs.image == raster, "y_geo"], GCPs.loc[GCPs.image == raster, "z_geo"]
    x_gr, y_gr, z_gr = geo.geocentric_cartesian_to_local_cartesian_coordinates(x_geo.values, y_geo.values, z_geo.values, lat_c *np.pi/180, lon_c *np.pi/180)
    GCPs.loc[GCPs.image == raster, ["x_gr", "y_gr", "z_gr"]] = np.array([x_gr, y_gr, z_gr]).T
    
    # fiducial coordinates
    xc, yc, alpha, delta_xi, delta_eta = all_params[raster]["internal_orientation"]["xc"], all_params[raster]["internal_orientation"]["yc"], all_params[raster]["internal_orientation"]["alpha"] * np.pi / 180, all_params[raster]["internal_orientation"]["delta_xi"], all_params[raster]["internal_orientation"]["delta_eta"]
    x_img, y_img = GCPs.loc[GCPs.image == raster, "x_img"], GCPs.loc[GCPs.image == raster, "y_img"]
    xi, eta = gio.image_to_fiducial_coordinates(x_img.values, y_img.values, xc, yc, alpha, delta_eta, delta_xi)
    GCPs.loc[GCPs.image == raster, ["xi", "eta"]] = np.array([xi, eta]).T
    
GCPs.head()

,image,lon,lat,h,x_img,y_img,x_geo,y_geo,z_geo,x_gr,y_gr,z_gr,xi,eta
0,DS09058A024MC012,15.669701,78.296340,475.462158,10090.231823,6378.932688,1.249830e+06,350598.250947,6.224197e+06,42286.552389,-67578.099573,-21.190105,64.344444,41.917789
1,DS09058A024MC012,15.822687,78.257831,902.182007,10106.594368,6190.345399,1.253023e+06,355105.791417,6.223741e+06,45902.009226,-71764.482684,335.057780,64.448778,40.678515
2,DS09058A024MC012,15.951993,78.240516,857.888794,10083.782353,6076.763062,1.254030e+06,358450.486485,6.223304e+06,48909.915218,-73591.109107,247.725214,64.303303,39.932126
3,DS09058A024MC012,16.027331,78.219240,833.358032,10101.936417,5980.737615,1.255788e+06,360739.850288,6.222796e+06,50713.696475,-75900.475625,182.167878,64.419065,39.301109
4,DS09058A024MC012,16.241150,78.218166,1070.018433,9995.579875,5857.839376,1.254592e+06,365470.086637,6.223003e+06,55592.640779,-75828.470156,379.179987,63.740838,38.493506


In [35]:
f = 76.2e-3
res = opt.least_squares(
    sr.space_rejection,
    x0=sr.pack_parameters(
        pp=[127./2, 127./2],
        ld_coeffs=[0., 0., 0., 0., 0., 0.],
        eo_params=[[0., 0., 3.22e5, 0., 0., -135 *np.pi/180][i] for _ in range(len(rasters)) for i in range(6)],
        n_img=len(rasters)
    ),
    args=(GCPs, f, len(rasters)),
    method="trf",
    x_scale="jac",
    max_nfev=1000,
)
print(res)

if res.success:
    pp, ld_coeffs, eo_params = sr.unpack_parameters(res.x, len(rasters))
    print(pp)
    print(ld_coeffs)
    print(eo_params)

# TODO: add bounds.

     message: `xtol` termination condition is satisfied.
     success: True
      status: 3
         fun: [ 1.474e+04]
           x: [ 7.482e+01  7.202e+01 ...  1.383e+00 -7.214e-01]
        cost: 108606384.40995772
         jac: [[-3.412e+02  3.331e+01 ...  1.622e+03  9.420e+03]]
        grad: [-5.029e+06  4.909e+05 ...  2.390e+07  1.388e+08]
  optimality: 9.115264822410914e+21
 active_mask: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
        nfev: 137
        njev: 131
[74.82392406096614, 72.02186705152751]
[-0.17685336929200823, -0.00010375101508231489, -7.294348397492905e-09, 4.221369978279563e-18, 0.0069197405870826505, 0.00121190549448971]
[-195919.1372803728, 72819.96414897944, 4423046.247675137, -1.1686364238774678, 1.7662760266384672, -3.846012839337351, -329.99705583385963, -417.1838859750654, 615908.8991134188, 0.6543867435083116, 1.3829554527892272, -0.7213885692203654]
